<a href="https://colab.research.google.com/github/RDXRUD/indian-constitution/blob/main/chat_with_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.7/389.7 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 11.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.me

In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00


In [4]:
!pip install torchvision

  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.3 MB/s eta 0:00:00
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0
    Uninstalling torch-2.4.0:
      Successfully uninstalled torch-2.4.0
ERROR: pip's dependency resolver does not

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

In [6]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

In [7]:
!mkdir pdfs

In [8]:
from langchain.document_loaders import PyPDFLoader
# loader = PyPDFDirectoryLoader("pdfs")
loader = PyPDFLoader("/content/10193_2022_16_1501_54754_Judgement_13-Aug-2024.pdf")
data = loader.load()

In [9]:
print(data)

[Document(metadata={'source': '/content/10193_2022_16_1501_54754_Judgement_13-Aug-2024.pdf', 'page': 0}, page_content='2024 INSC 602\n \n1                             \nREPORTABLE\nIN THE SUPREME COURT OF INDIA\nCRIMINAL APPELLATE JURISDICTION\nCRIMINAL   APPEAL NO.  3369   OF 2024\n[  @ SPECIAL LEAVE PETITION (CRL.) NO.4022 OF 2022  ]\nSRI SUJIES BENEFIT FUNDS LIMITED                           …APPELLANT\nVERSUS\nM. JAGANATHUAN        … RESPONDENT\nJ U D G M E N T\nAHSANUDDIN AMANULLAH, J.\nHeard Mr B. Ragunath, learned counsel for the appellant and\nMr. S. Nagamuthu, learned senior counsel for the respondent.\n2.Leave granted.\n3. The  present  appeal  arises  out  of  the  Final  Judgment  dated\n29.01.2020  (hereinafter  referred  to  as  the  “ impugned  judgment”),\npassed by the learned Single Judge of the High Court of Judicature at'), Document(metadata={'source': '/content/10193_2022_16_1501_54754_Judgement_13-Aug-2024.pdf', 'page': 1}, page_content=' \n2                      

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [11]:
docs = text_splitter.split_documents(data)


In [12]:
print(len(docs))


44


In [13]:

docs[3]

Document(metadata={'source': '/content/10193_2022_16_1501_54754_Judgement_13-Aug-2024.pdf', 'page': 1}, page_content='“accused”),  being  a  subscriber  of  the  Appellant-chitfund  company\n(hereinafter  also  referred  to  as  the  “complainant”),  borrowed  loan\namounts on several dates from the Appellant over a period of about two\nyears which swelled to a sum of Rs.21,09,000/- (Rupees Twenty One\nLakhs and Nine Thousand) including interest, after eight years. The\nloans were advanced in the following manner: Rs.1,50,000/- (Rupees\nOne Lakh and Fifty Thousand) was given on 09.03.1995; Rs.6,00,000/-')

In [14]:
docs[4]

Document(metadata={'source': '/content/10193_2022_16_1501_54754_Judgement_13-Aug-2024.pdf', 'page': 1}, page_content='(Rupees Six Lakhs) on 29.12.1995; Rs.1,00,000/- (Rupees One Lakh)\non 22.03.1995; Rs.3,00,000/- (Rupees Three Lakhs) on 11.03.1996;\nRs.1,00,000/-  (Rupees  One  Lakh)  on  09.04.1997;  and  finally,\nRs.2,00,000/- (Rupees Two Lakhs) on 24.04.1997. In order to partly\ndischarge  the  aforesaid  loan  amounts,  Cheque  No.0150573  dated\n03.02.2003 was issued by the accused for a sum of Rs.19,00,000/-')

In [15]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 6.4 MB/s eta 0:00:00


In [17]:
from langchain.vectorstores import Chroma

In [18]:
vectordb=Chroma.from_documents(docs,embedding=embeddings, persist_directory='./data')


In [19]:

vectordb.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [25]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
import torch

In [26]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              #load_in_8bit=True,
                                              load_in_4bit=True
                                             )
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")

In [27]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30

              )

In [28]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [29]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7dd659f4d390>, model_kwargs={'temperature': 0})

In [41]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end."""

In [42]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"

In [43]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS


In [44]:

instruction = """
{context}

Question: {question}
"""

In [45]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST


In [35]:
template


"[INST]<>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [46]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [47]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [50]:
# result = qa_chain("borrowed loan amounts on several dates from the Appellant over a period of about two years which swelled to")
result = qa_chain("benefit of doubt given?")



In [51]:
result['result']

'[INST]<>\nUse the following pieces of context to answer the question at the end.\n<>\n\n\n6                             \nevidence was initially not done for no justifiable/valid reason before the\nlearned Trial Court and, even upon the plea for adducing additional\nevidence under Section 391 of the Code, the presumption has not been\ndislodged  as  required  under  law,  and  still  the  accused  has  been\nacquitted.\n9.Learned counsel submitted that the Appellate Court has given\nbenefit of doubt to the respondent by raising question about the figure\n\nit were an inquiry.’\n\nQuestion: benefit of doubt given?\n[/INST]  Based on the context provided, the answer to the question is "yes". The learned counsel for the appellant has argued that the Appellate Court has given the benefit of doubt to the respondent by raising questions about the figure in the evidence. This suggests that the Appellate Court was not convinced beyond reasonable doubt of the accused\'s guilt, and therefore ga

AttributeError: 'Chroma' object has no attribute 'get_all'